In [ ]:
# ==============================================================================
# IMPORTAÇÕES E CONFIGURAÇÕES GLOBAIS
# ==============================================================================
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from matplotlib.lines import Line2D

plt.style.use('bmh')
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['figure.dpi'] = 100

# Análise 1: Dinâmica Histórica do Câmbio BRL/USD (2007-2025)

Esta seção foca no desempenho histórico do câmbio `BRLUSD=X` (Real por Dólar), analisando a volatilidade, o retorno acumulado e as trajetórias anuais.

In [ ]:
# ==============================================================================
# DOWNLOAD DOS DADOS - POST 1
# ==============================================================================
# Baixamos os dados históricos completos do BRL/USD uma única vez.
print("Baixando dados para BRLUSD=X (Post 1)...")
brl_historico = yf.download('BRLUSD=X', start='2007-01-01', end=datetime.now())
brl_historico.dropna(inplace=True)
print("Dados baixados com sucesso.")

### 1.1 Análise YTD: Comparando períodos iguais

Para uma comparação justa, calculamos a volatilidade e o retorno de **todos os anos** usando apenas os dados desde 1º de Janeiro até o dia atual (12 de Novembro).

In [ ]:
# ==============================================================================
# CÁLCULOS YTD (VOLATILIDADE E RETORNO)
# ==============================================================================
print("Calculando análise YTD...")

# --- LÓGICA DE COMPARAÇÃO DE PERÍODOS IGUAIS ---
# Usamos o dayofyear para obter um número (ex: 316 para 12/Nov)
dia_limite = pd.to_datetime('2025-11-12').dayofyear
# Filtra os dados de TODOS os anos para incluir apenas até esse "dia do ano"
brl_periodo_comparavel = brl_historico[brl_historico.index.dayofyear <= dia_limite].copy()
print(f"Análise sendo feita para todos os anos, considerando dados somente até o dia {dia_limite} de cada ano.")

# --- CÁLCULOS SOBRE OS DADOS FILTRADOS ---
# 3. Cálculo da Volatilidade no período
brl_periodo_comparavel['RetornoDiario'] = brl_periodo_comparavel['Close'].pct_change()
vol_brl_anualizada = brl_periodo_comparavel.groupby(brl_periodo_comparavel.index.year)['RetornoDiario'].std() * np.sqrt(252)
vol_brl_anualizada *= 100

# 4. Cálculo do Retorno no período
primeiro_preco_ano = brl_periodo_comparavel.groupby(brl_periodo_comparavel.index.year)['Close'].first()
ultimo_preco_periodo = brl_periodo_comparavel.groupby(brl_periodo_comparavel.index.year)['Close'].last()
retorno_no_periodo = (ultimo_preco_periodo / primeiro_preco_ano - 1) * 100

# 5. Tabela Consolidada
analise_ytd = pd.DataFrame({
    'Volatilidade (%)': vol_brl_anualizada.squeeze(),
    'Retorno YTD (%)': retorno_no_periodo.squeeze()
}).dropna()

# Imprime os dados atuais para o post
try:
    print(f"\nDados de 2025 (Volatilidade e Retorno YTD):\n{analise_ytd.loc[2025]}")
except KeyError:
    print("\nDados de 2025 ainda não disponíveis na análise YTD.")
print(f"\nMédia da Volatilidade (Período Comparável): {analise_ytd['Volatilidade (%)'].mean():.2f}%")

In [ ]:
# ==============================================================================
# GRÁFICO DE BARRAS (VOLATILIDADE E RETORNO YTD)
# ==============================================================================
#@title Retorno do câmbio USDBRL - 01/jan a 12/nov, de 2007 a 2025
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(12, 5))
analise_ytd['Volatilidade (%)'].plot(kind='bar', ax=axes[0], color='c', edgecolor='black')
axes[0].set_title('Volatilidade Anualizada do câmbio USD/BRL - 01/Jan a 12/Nov', fontsize=16)
axes[0].set_ylabel('Volatilidade (%)')

retornos = analise_ytd['Retorno YTD (%)']
cores = ['olive' if x >= 0 else 'coral' for x in retornos]
retornos.plot(kind='bar', ax=axes[1], color=cores, edgecolor='black')
axes[1].axhline(0, color='black', linewidth=0.8)
axes[1].set_title('Retorno Acumulado do câmbio USD/BRL - 01/Jan a 12/Nov', fontsize=16)
axes[1].set_ylabel('Retorno (%)')
axes[1].set_xlabel('Ano')
plt.tight_layout()
plt.savefig('results\\post1_bars.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ==============================================================================
# GRÁFICO DE DISPERSÃO (RISCO VS. RETORNO)
# ==============================================================================
#@title Risco x Retorno câmbio BRLUSD - 01/jan a 12/nov, de 2007 a 2025
fig, ax = plt.subplots(figsize=(14, 6))
ax.scatter(x=analise_ytd['Volatilidade (%)'], y=analise_ytd['Retorno YTD (%)'], s=80, alpha=0.7, edgecolor='black')
for ano, row in analise_ytd.iterrows():
    ax.annotate(ano, (row['Volatilidade (%)'] + 0.3, row['Retorno YTD (%)']))
ax.axhline(0, color='black', linestyle='-')
media_vol = analise_ytd['Volatilidade (%)'].mean()
ax.axvline(media_vol, color='red', linestyle='--', label=f'Média de Volatilidade ({media_vol:.1f}%)')

ax.set_title('Análise de Risco vs. Retorno (Período de Jan a Nov)', fontsize=16)
ax.set_xlabel('Risco (Volatilidade %)')
ax.set_ylabel('Retorno Acumulado (%)')
ax.legend()
plt.tight_layout()
plt.savefig('results\post1_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

### 1.2 Dinâmica dos Melhores e Piores Anos (com Ano Atual)

Para esta análise, comparamos a trajetória de todos os anos com base no **dia de pregão sequencial**. Isso nos permite sobrepor o desempenho do ano atual (linha azul) contra os 3 melhores (verde), os 3 piores (vermelho) e todos os outros (cinza).

In [ ]:
# ==============================================================================
# GRÁFICO MELHORES/PIORES/ANO ATUAL
# ==============================================================================
#@title Dinâmica do BRL/USD: Melhores, Piores e Ano Atual (2007-2025)

# 2. Pivotar os Dados por Dias de Pregão (usando brl_historico da Célula 3)
df_temp_c11 = pd.DataFrame(brl_historico['Close']) # Usando a coluna 'Close'
df_temp_c11.columns = ['Preco'] # Renomeia para evitar conflito
df_temp_c11['Ano'] = df_temp_c11.index.year
df_temp_c11['DiaDePregao'] = df_temp_c11.groupby('Ano').cumcount() + 1
dados_pivotados_c11 = df_temp_c11.pivot_table(index='DiaDePregao', columns='Ano', values='Preco')

# 3. Excluir Dias Nulos e Construir o Número-Índice
# dropna() garante a comparação de períodos iguais
dados_comuns_c11 = dados_pivotados_c11.dropna()
numero_indice_c11 = dados_comuns_c11.apply(lambda coluna: (coluna / coluna.iloc[0]) * 100)

# 4. Identificar os 3 melhores e 3 piores anos
ultimo_preco_ano_c11 = dados_comuns_c11.iloc[-1]
primeiro_preco_ano_c11 = dados_comuns_c11.iloc[0]
retorno_anual_c11 = (ultimo_preco_ano_c11 / primeiro_preco_ano_c11 - 1)

ano_atual_int = datetime.now().year
melhores_anos = retorno_anual_c11.nlargest(3)
piores_anos = retorno_anual_c11.nsmallest(3)

# 5. Plotar o Gráfico
fig, ax = plt.subplots(figsize=(12, 5))

cores_melhores = sns.color_palette("viridis", n_colors=3)
cores_piores = sns.color_palette("autumn_r", n_colors=3)
cor_ano_atual = 'tomato' # Cor de destaque
legend_melhores, legend_piores, legend_atual = {}, {}, {}

# Listas dos anos
todos_os_anos = numero_indice_c11.columns
melhores_anos_list = list(melhores_anos.index)
piores_anos_list = list(piores_anos.index)
anos_destaque_set = set(melhores_anos_list + piores_anos_list + [ano_atual_int])
anos_cinza = [ano for ano in todos_os_anos if ano not in anos_destaque_set]

# Plota todos os outros anos em cinza
for ano in anos_cinza:
    linha = numero_indice_c11[ano]
    ax.plot(linha.index, linha, color='dimgray', linewidth=1.0, alpha=0.7, label='_nolegend_')

# Plota os anos de baixo desempenho (Real fraco)
for i, ano in enumerate(piores_anos_list):
    if ano == ano_atual_int: continue
    linha = numero_indice_c11[ano]
    p, = ax.plot(linha.index, linha, color=cores_piores[i], linewidth=2.0, linestyle='--')
    legend_piores[ano] = p

# Plota os anos de alto desempenho (Real forte)
for i, ano in enumerate(melhores_anos_list):
    if ano == ano_atual_int: continue
    linha = numero_indice_c11[ano]
    p, = ax.plot(linha.index, linha, color=cores_melhores[i], linewidth=2.5)
    legend_melhores[ano] = p

# Plota o Ano Atual (por último)
if ano_atual_int in numero_indice_c11.columns:
    linha_atual = numero_indice_c11[ano_atual_int]
    p_atual, = ax.plot(linha_atual.index, linha_atual, color=cor_ano_atual, linewidth=3.0, linestyle='-')
    legend_atual[f"{ano_atual_int}"] = p_atual

ax.axhline(100, color='black', linestyle='-.', linewidth=1)

# Criação das Legendas Separadas
leg1 = ax.legend(legend_melhores.values(), legend_melhores.keys(),
                 title="Top valorizações", loc='upper left')
leg2 = ax.legend(legend_piores.values(), legend_piores.keys(),
                 title="Top desvalorizações", loc='lower left')
if legend_atual:
    leg3 = ax.legend(legend_atual.values(), legend_atual.keys(),
                     loc='upper center',
                     facecolor='white', framealpha=0.8)
    ax.add_artist(leg3)

ax.add_artist(leg1)
ax.add_artist(leg2)

ax.set_title(f'Dinâmica do câmbio USD/BRL: melhores, piores e ano atual (2007-{ano_atual_int})', fontsize=16)
ax.set_ylabel('Número-Índice (Base: 1º Pregão do Ano = 100)', fontsize=12)
ax.set_xlabel('Dias de Pregão Sequenciais no Ano', fontsize=12)

plt.tight_layout()
plt.savefig('results\post1_best_worst_atual.png', dpi=150, bbox_inches='tight')
plt.show()